In [ ]:
from scripts.ocr_utils import html_to_text, ocr_image_file, ocr_pdf_file, extract_basic_fields

# ZIP upload + extract
from google.colab import files
uploaded = files.upload()
zip_path = list(uploaded.keys())[0]
EXTRACT_DIR = Path("financial_dataset")
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(EXTRACT_DIR)

# Surinkti failus
html_files = list(EXTRACT_DIR.rglob("*.html"))
pdf_files = list(EXTRACT_DIR.rglob("*.pdf"))
image_files = []
for ext in ["png","jpg","jpeg","tiff","bmp"]:
    image_files += list(EXTRACT_DIR.rglob(f"*.{ext}"))

rows = []
# Process HTML
for f in tqdm(html_files):
    text = html_to_text(f.read_text(encoding='utf-8', errors='ignore'))
    rows.append({"file": str(f), "text": text, "source_type": "html", **extract_basic_fields(text)})

# Process PDFs
for f in tqdm(pdf_files):
    try:
        text = ocr_pdf_file(f)
        rows.append({"file": str(f), "text": text, "source_type": "pdf", **extract_basic_fields(text)})
    except Exception as e:
        print("PDF OCR error:", f, e)

# Process Images
for f in tqdm(image_files):
    try:
        text = ocr_image_file(f)
        rows.append({"file": str(f), "text": text, "source_type": "image", **extract_basic_fields(text)})
    except Exception as e:
        print("Image OCR error:", f, e)

df = pd.DataFrame(rows)
df.to_csv("dataset.csv", index=False)
